In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error as mse
import lightgbm
from lightgbm import LGBMRegressor

In [2]:
train_df = pd.read_csv("../data/Train.csv")
test_df = pd.read_csv("../data/Test.csv")
ss = pd.read_csv("../data/SampleSubmission.csv")

In [3]:
validation=0

In [4]:
target = "target"

In [5]:
train_df["xx"]=train_df.groupby(target)[target].transform("count")
train_df=train_df[train_df["xx"]>1]
train_df=train_df.drop("xx",axis=1)
train_df["target"].nunique()

1481

In [6]:
train_df[target].value_counts()

101000000.0    103
141000000.0     94
102000000.0     88
107000000.0     87
110000000.0     87
              ... 
2120000.0        2
9730000.0        2
5150000.0        2
8360000.0        2
9970000.0        2
Name: target, Length: 1481, dtype: int64

In [7]:
train_df["device"].unique()

array(['pc1', 'pc3'], dtype=object)

In [8]:
test_df["device"].unique()

array(['pc2', 'pc4'], dtype=object)

In [9]:
train_df["timestamp"].value_counts()

1624546748    2
1624551351    2
1624547690    2
1624547622    2
1624551307    2
             ..
1624367902    1
1624542101    1
1624369420    1
1624368572    1
1624376571    1
Name: timestamp, Length: 27197, dtype: int64

In [10]:
train_df.corr().abs()[target].sort_values(ascending=False)

target                          1.000000
SCell_SNR_max                   0.642068
SCell_Downlink_Average_MCS      0.627163
PCell_SNR_max                   0.572684
PCell_RSSI_max                  0.503193
SCell_RSRP_max                  0.503059
SCell_Downlink_Num_RBs          0.483362
PCell_RSRP_max                  0.472979
SCell_RSSI_max                  0.452446
PCell_Downlink_Average_MCS      0.444546
operator                        0.431136
PCell_Cell_Identity             0.413958
PCell_Downlink_Num_RBs          0.403038
SCell_RSRQ_max                  0.289486
PCell_Downlink_bandwidth_MHz    0.251947
temperature                     0.249340
apparentTemperature             0.249340
cloudCover                      0.248666
dewPoint                        0.246258
humidity                        0.230337
timestamp                       0.201864
PCell_RSRQ_max                  0.191119
pressure                        0.181916
precipProbability               0.180588
SCell_Cell_Ident

In [11]:
test_df["area"]=test_df["area"].replace({'Avenue':0, 'Park':1, 'Highway':2, 'Residential':3, 'Tunnel':4})
train_df["area"]=train_df["area"].replace({'Avenue':0, 'Park':1, 'Highway':2, 'Residential':3, 'Tunnel':4})
test_df.device=test_df.device.str[-1].astype("int")
train_df.device=train_df.device.str[-1].astype("int")

In [12]:
if validation==0:
    test_df[target]=np.nan
    test_df=test_df[train_df.columns]
    train_df=train_df.append(test_df)
else:
    test_df=train_df[train_df.device==3]
    ss = test_df[ss.columns]
    train_df=train_df[train_df.device==1]
    test_df[target]=np.nan
    test_df=test_df[train_df.columns]
    train_df=train_df.append(test_df)

In [13]:
from datetime import datetime
def fix_time(old_time):
    new_time=datetime.fromtimestamp(old_time)
    return new_time

train_df["hour"]=train_df.timestamp.apply(fix_time)
train_df["hour"]=train_df["hour"].astype(str).str[11:13].astype("int")


In [14]:
train_df

,id,timestamp,device,PCell_RSRP_max,PCell_RSRQ_max,PCell_RSSI_max,PCell_SNR_max,PCell_Downlink_Num_RBs,PCell_Downlink_Average_MCS,PCell_Downlink_bandwidth_MHz,...,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility,Traffic Jam Factor,area,target,hour
0,Id_qt2qn56050,1624367008,1,-84.498750,-14.586875,-49.855625,16.457,35076.0,23.0,20.0,...,0.62,1013.6,2.39,0.95,4.0,16.093,0.38979,3,51500000.0,14
1,Id_f1zf07cwb6,1624372465,3,-86.818125,-11.982500,-54.838125,16.674,42704.0,24.0,20.0,...,0.59,1013.5,2.29,0.82,3.0,16.093,3.78322,3,50300000.0,15
2,Id_uoxdz7di5b,1624371871,3,-101.916250,-13.257500,-71.241250,9.392,36017.0,22.0,20.0,...,0.61,1013.8,2.13,0.82,3.0,16.093,2.08425,3,16500000.0,15
3,Id_hzn8nj29gu,1624542066,1,-95.598750,-14.696250,-61.660000,10.726,95955.0,17.0,20.0,...,0.71,1017.9,1.97,0.99,3.0,16.093,8.80702,0,46400000.0,14
4,Id_2kfrccrfiw,1624542798,1,-97.735000,-14.867500,-63.170625,10.889,83062.0,19.0,20.0,...,0.71,1017.9,1.97,0.99,3.0,16.093,2.21843,1,50200000.0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18238,Id_thgt73179f,1624370207,2,-81.506250,-9.556875,-78.813125,24.005,49881.0,28.0,20.0,...,0.62,1013.8,2.12,0.84,3.0,16.093,1.82751,1,NaN,14
18239,Id_r6wd5ofvtt,1624371137,2,-84.858750,-12.547500,-83.305000,11.985,29323.0,22.0,20.0,...,0.62,1013.9,2.12,0.83,3.0,16.093,3.14159,1,NaN,15
18240,Id_p6sqas9rgd,1624376766,2,-82.406875,-12.343125,-79.631875,12.719,37837.0,21.0,20.0,...,0.57,1013.7,2.19,0.76,2.0,16.093,1.57307,3,NaN,16
18241,Id_yre3gwygul,1624353986,4,-95.212500,-10.181875,-65.595625,11.800,47277.0,15.0,20.0,...,0.73,1012.2,4.23,0.97,4.0,16.093,0.51020,1,NaN,10


In [15]:
test_df

,id,timestamp,device,PCell_RSRP_max,PCell_RSRQ_max,PCell_RSSI_max,PCell_SNR_max,PCell_Downlink_Num_RBs,PCell_Downlink_Average_MCS,PCell_Downlink_bandwidth_MHz,...,dewPoint,humidity,pressure,windSpeed,cloudCover,uvIndex,visibility,Traffic Jam Factor,area,target
0,Id_ln0e0hfrgx,1624374179,2,-97.121875,-12.923750,-87.554375,13.293,39276.0,24.0,20,...,14.55,0.59,1013.6,2.27,0.83,3.0,16.093,3.07702,0,NaN
1,Id_svf7nz9fxv,1624366264,4,-65.202500,-9.944375,-35.501875,21.823,39566.0,25.0,20,...,13.92,0.62,1013.6,2.42,0.95,4.0,16.093,0.66000,1,NaN
2,Id_ww2mh07gwj,1624375795,2,-79.298125,-17.468750,-74.823750,24.114,41900.0,27.0,20,...,14.65,0.60,1013.5,2.26,0.85,3.0,16.093,2.49330,1,NaN
3,Id_v88r4y03ww,1624374065,2,-110.128125,-13.513750,-92.566875,6.282,76310.0,18.0,20,...,14.45,0.59,1013.6,2.27,0.82,3.0,16.093,5.74454,2,NaN
4,Id_2u4y4kzglh,1624374361,2,-117.451250,-18.753125,-93.048125,-5.767,44887.0,7.0,20,...,14.55,0.59,1013.6,2.27,0.83,3.0,16.093,3.48651,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18238,Id_thgt73179f,1624370207,2,-81.506250,-9.556875,-78.813125,24.005,49881.0,28.0,20,...,14.77,0.62,1013.8,2.12,0.84,3.0,16.093,1.82751,1,NaN
18239,Id_r6wd5ofvtt,1624371137,2,-84.858750,-12.547500,-83.305000,11.985,29323.0,22.0,20,...,14.69,0.62,1013.9,2.12,0.83,3.0,16.093,3.14159,1,NaN
18240,Id_p6sqas9rgd,1624376766,2,-82.406875,-12.343125,-79.631875,12.719,37837.0,21.0,20,...,14.78,0.57,1013.7,2.19,0.76,2.0,16.093,1.57307,3,NaN
18241,Id_yre3gwygul,1624353986,4,-95.212500,-10.181875,-65.595625,11.800,47277.0,15.0,20,...,13.92,0.73,1012.2,4.23,0.97,4.0,16.093,0.51020,1,NaN


In [16]:
del test_df

In [17]:
drop_cols=['id','device', 'target']

cols=list(train_df.drop(drop_cols,axis=1).columns)[:15]
for a, col1 in enumerate(cols):
    for b, col2 in enumerate(cols):
        if a<b:
            train_df[col1+col2]=train_df[col1]*train_df[col2]
            
train_df["area_mean"] = train_df.groupby("PCell_Cell_Identity")[target].transform("mean")
test_df=train_df[train_df[target].isna()]
train_df=train_df[train_df[target].notna()]
train_cols=list(train_df.drop(drop_cols,axis=1).columns)
#train_cols = ccc[:50]
train_df=train_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
test_df=test_df.replace({np.inf:np.nan,(-1*np.inf):np.nan})
my_model=LGBMRegressor(objective="tweedie")#CatBoostRegressor(silent=True,learning_rate=0.1, n_estimators=10)
my_model.fit(train_df[train_cols],train_df[target])
predictions = my_model.predict(test_df[train_cols])
submission = pd.DataFrame(data=test_df["id"], columns = ["id"])
submission["target"] = predictions
if validation==0:
    df1=pd.merge(ss[['id']],submission,on='id',how='left')
    df1.to_csv('../predictions/prdiction_4.csv',index=False)
else:
    df1=pd.merge(ss,submission,on='id',how='left')
    print((mse(df1["target_x"],df1["target_y"])**0.5)/1000000)
df1

,id,target
0,Id_hg4i94kfbg,2.846770e+07
1,Id_j1afeth58t,2.272353e+07
2,Id_bahkipht8j,2.206795e+07
3,Id_t1iry5orn0,2.143091e+07
4,Id_dnlh2w774v,2.172527e+07
...,...,...
18238,Id_t4ukpeaxa5,4.851436e+07
18239,Id_dallii8q6q,4.376009e+07
18240,Id_p0neksuxwp,1.048648e+08
18241,Id_s2yku2tuvb,9.809054e+07


In [18]:
x=sorted(zip(map(lambda x: round(x, 4), my_model.feature_importances_),train_cols), reverse=True)
x = pd.DataFrame(x)
x[1].values

array(['PCell_SNR_maxPCell_Downlink_Num_RBs', 'area_mean',
       'PCell_Downlink_Num_RBsPCell_Downlink_Average_MCS', 'Latitude',
       'Longitude', 'SCell_Downlink_Average_MCS',
       'PCell_RSRQ_maxPCell_Downlink_Average_MCS', 'speed_kmh',
       'timestampPCell_freq_MHz', 'COG',
       'SCell_SNR_maxSCell_Downlink_Num_RBs',
       'PCell_RSRQ_maxPCell_SNR_max',
       'timestampPCell_Downlink_bandwidth_MHz',
       'PCell_Downlink_Num_RBsSCell_SNR_max',
       'timestampPCell_Downlink_Average_MCS',
       'PCell_Downlink_Average_MCSSCell_Downlink_Num_RBs', 'Altitude',
       'PCell_SNR_maxSCell_Downlink_Num_RBs', 'timestamp',
       'PCell_Downlink_Average_MCSPCell_Cell_Identity',
       'PCell_RSRQ_maxPCell_Downlink_bandwidth_MHz',
       'PCell_Cell_IdentitySCell_Downlink_Num_RBs',
       'PCell_RSSI_maxPCell_Downlink_Average_MCS',
       'PCell_RSSI_maxPCell_Cell_Identity',
       'PCell_SNR_maxPCell_Cell_Identity',
       'PCell_RSRQ_maxSCell_Downlink_Num_RBs', 'temperature',
